<a href="https://colab.research.google.com/github/sarwaridas/DRL2/blob/main/microsoft_reccomenders/Microsoft_Reccomender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Final Project

/content/drive/.shortcut-targets-by-id/1IDZheLhbUxf4Hl-NziHhCULeeSsjfiaL/Final Project


In [2]:
# !git clone https://github.com/microsoft/recommenders.git

In [5]:
%cd recommenders

/content/drive/.shortcut-targets-by-id/1IDZheLhbUxf4Hl-NziHhCULeeSsjfiaL/Final Project/recommenders


In [6]:
!ls

AUTHORS.md	    CONTRIBUTING.md  LICENSE	     README.md	   SETUP.md
CODE_OF_CONDUCT.md  docs	     MANIFEST.in     recommenders  setup.py
conda.md	    examples	     NEWS.md	     scenarios	   tests
contrib		    GLOSSARY.md      pyproject.toml  SECURITY.md   tools


In [141]:
from recommenders.models import sar
import pandas as pd
import numpy as np

In [142]:
### Update path to Dignitica data sample_session.df file here:
digniticaDatapath = '/content/drive/MyDrive/sampled_sessions.df'

In [143]:
df = pd.read_pickle(digniticaDatapath)
df = df[df['is_buy'] == 1]

In [144]:
df.head()

,session_id,timestamp,item_id,is_buy
420,150,2016-05-06,329,1
440,156,2016-05-27,336,1
660,246,2016-05-09,514,1
740,312,2016-04-25,577,1
741,312,2016-04-25,574,1


In [145]:
df['timestamp'] = pd.to_datetime(df['timestamp'],format= '%Y-%m-%d' ).values.astype(np.int64) // 10 ** 9 + df.index

In [146]:
df['rating'] = df['is_buy']

In [147]:
df.head()

,session_id,timestamp,item_id,is_buy,rating
420,150,1462493220,329,1,1
440,156,1464307640,336,1,1
660,246,1462752660,514,1,1
740,312,1461543140,577,1,1
741,312,1461543141,574,1,1


In [148]:
sars = sar.SAR('session_id','item_id','is_buy','timestamp', timedecay_formula=True)

In [149]:
sars.fit(df)

In [165]:
prediction = sars.recommend_k_items(df,8)

In [155]:
import recommenders.evaluation.python_evaluation as evalMetrics

prediction3 = sars.recommend_k_items(df,3)
prediction5 = sars.recommend_k_items(df,5)
prediction8 = sars.recommend_k_items(df,8)
ndcg3 = evalMetrics.ndcg_at_k(df, prediction3,'session_id','item_id','prediction', k = 3)
ndcg5 = evalMetrics.ndcg_at_k(df, prediction5,'session_id','item_id','prediction', k = 5)
ndcg8 = evalMetrics.ndcg_at_k(df, prediction8,'session_id','item_id','prediction', k = 8)

print('NDCG at 3: ' + str(ndcg3))
print('NDCG at 5: ' + str(ndcg5))
print('NDCG at 8: ' + str(ndcg8))

NDCG at 3: 1.0566246603605576
NDCG at 5: 1.0382341897376748
NDCG at 8: 1.0293221965259156


In [166]:
reccomendations = prediction.groupby('session_id')['item_id'].apply(list).reset_index(name='reccomended')
reccomendations = reccomendations.reset_index()

In [168]:
reccomendations.head()

,index,session_id,reccomended
0,0,150,"[329, 21027, 17594, 659, 3963, 42726, 1743, 17..."
1,1,156,"[336, 2756, 1743, 42758, 4894, 22389, 13055, 4..."
2,2,246,"[514, 1743, 4894, 42758, 13084, 42725, 13055, ..."
3,3,312,"[574, 577, 42758, 13084, 13055, 42726, 42725, ..."
4,4,322,"[594, 42726, 4894, 1743, 42758, 13084, 42725, ..."


In [169]:
reccomendations.loc[reccomendations['session_id']==150].reccomended[0]

[329, 21027, 17594, 659, 3963, 42726, 1743, 17740]

In [170]:
### Hit rate calculations:
hits3 = 0
total3 = 0
hits5 = 0
total5 = 0
hits8 = 0
total8 = 0
for index, row in df.iterrows():
  if index % 2000 == 0:
    print('Progress = ' + str(index/len(df.index)))
  reccomendedVales = reccomendations.loc[reccomendations['session_id']==row['session_id']].reccomended.values[0]
  if row['item_id'] in reccomendedVales[:3]:
    hits3 += 1
    total3 += 1
  else:
    total3 += 1
  if row['item_id'] in reccomendedVales[:5]:
    hits5 += 1
    total5 += 1
  else:
    total5 += 1
  if row['item_id'] in reccomendedVales[:8]:
    hits8 += 1
    total8 += 1
  else:
    total8 += 1
print('Hit rate at 3'+ ':' + str(hits3/total3))
print('Hit rate at 5'+ ':' + str(hits5/total5))
print('Hit rate at 8'+ ':' + str(hits8/total8))

Progress = 0.0
Progress = 0.13658403332650412
Progress = 0.27316806665300825
Progress = 0.4097520999795124
Progress = 0.5463361333060165
Progress = 0.6829201666325206
Progress = 0.8195041999590248
Progress = 0.9560882332855289
Hit rate at 3:0.9352591682032371
Hit rate at 5:0.9786928908010654
Hit rate at 8:0.9941268865669604


#### Retail Rocket

#### Data setup

In [46]:
 ! pip install -q kaggle

In [47]:
%cd /content

/content


In [48]:
 ! mkdir ~/.kaggle

In [49]:
## Upload kaggle.json file before running this block to get dataset
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [50]:
! kaggle datasets download -d retailrocket/ecommerce-dataset
! unzip ecommerce-dataset.zip -d retailrocket

100% 290M/291M [00:05<00:00, 97.7MB/s]
100% 291M/291M [00:05<00:00, 58.2MB/s]
Archive:  ecommerce-dataset.zip
  inflating: retailrocket/category_tree.csv  
  inflating: retailrocket/events.csv  
  inflating: retailrocket/item_properties_part1.csv  
  inflating: retailrocket/item_properties_part2.csv  


In [51]:
import time
import numpy as np
import pandas as pd

import multiprocessing as mp
from multiprocessing import Pool
from functools import partial

In [52]:
df = pd.read_csv("retailrocket/events.csv")
np.unique(df.event.to_numpy())

array(['addtocart', 'transaction', 'view'], dtype=object)

In [53]:
df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [54]:
import os
from sklearn.preprocessing import LabelEncoder
data_directory = 'retailrocket'
event_df = pd.read_csv(os.path.join(data_directory, 'events.csv'), header=0)
event_df.columns = ['timestamp','session_id','behavior','item_id','transid']
###remove transid column
event_df =event_df[event_df['transid'].isnull()]
event_df = event_df.drop('transid',axis=1)
##########remove users with <=2 interactions
event_df['valid_session'] = event_df.session_id.map(event_df.groupby('session_id')['item_id'].size() > 2)
event_df = event_df.loc[event_df.valid_session].drop('valid_session', axis=1)
##########remove items with <=2 interactions
event_df['valid_item'] = event_df.item_id.map(event_df.groupby('item_id')['session_id'].size() > 2)
event_df = event_df.loc[event_df.valid_item].drop('valid_item', axis=1)
######## transform to ids
item_encoder = LabelEncoder()
session_encoder= LabelEncoder()
behavior_encoder=LabelEncoder()
event_df['item_id'] = item_encoder.fit_transform(event_df.item_id)
event_df['session_id'] = session_encoder.fit_transform(event_df.session_id)
event_df['behavior']=behavior_encoder.fit_transform(event_df.behavior)
###########sorted by user and timestamp
event_df['is_buy']=1-event_df['behavior']
event_df = event_df.drop('behavior', axis=1)
sorted_events = event_df.sort_values(by=['session_id', 'timestamp'])

In [55]:
sorted_events = sorted_events[sorted_events['is_buy'] == 1]

In [56]:
sorted_events.head()

,timestamp,session_id,item_id,is_buy
1133190,1440914628202,2,9750,1
138668,1433719818230,19,68766,1
1357916,1441951937583,20,68635,1
1368056,1441951967871,20,27419,1
1367869,1441952264613,20,6210,1


In [58]:
sorted_events['rating'] = sorted_events['is_buy']

In [59]:
sarsRetail = sar.SAR('session_id','item_id','is_buy','timestamp',timedecay_formula=True)

In [60]:
sorted_events = sorted_events.drop_duplicates()

In [61]:
sorted_events.sort_values(['timestamp'])

,timestamp,session_id,item_id,is_buy,rating
1462974,1430622004384,96423,45296,1,1
1463435,1430622029427,96423,45296,1,1
1465262,1430622085008,136274,51456,1,1
1463439,1430622240603,36191,18997,1,1
1465260,1430622602006,44393,52119,1,1
...,...,...,...,...,...
1456546,1442544102108,21274,3212,1,1
1459656,1442544320572,154193,20334,1,1
1456746,1442544322842,154193,20334,1,1
1454076,1442544649803,79612,28344,1,1


In [62]:
sorted_events[sorted_events.duplicated() == True]

,timestamp,session_id,item_id,is_buy,rating


In [108]:
sorted_events = sorted_events.reset_index()

In [63]:
sarsRetail.fit(sorted_events)

In [64]:
usersdf = sorted_events.drop_duplicates(subset = "session_id")

In [80]:
usersdf

,timestamp,session_id,item_id,is_buy,rating
1133190,1440914628202,2,9750,1,1
138668,1433719818230,19,68766,1,1
1357916,1441951937583,20,68635,1,1
848067,1439599816912,24,1436,1,1
1413091,1442290183293,25,892,1,1
...,...,...,...,...,...
667959,1438729535724,195468,54945,1,1
2325662,1436804352069,195471,4305,1,1
1164581,1441098006805,195501,28748,1,1
466922,1435098167672,195503,59333,1,1


In [127]:
## Chunking reccomendations

reccomendationsRetail = pd.DataFrame(columns = ["session_id", "reccomended"])
startidx = 0
endidx = 1000
while startidx < len(usersdf.index):
  batch = sarsRetail.recommend_k_items(usersdf[startidx:endidx],8).groupby('session_id')['item_id'].apply(list).reset_index(name='reccomended')
  reccomendationsRetail = pd.concat([reccomendationsRetail, batch])
  startidx = endidx
  endidx += 1000

In [128]:
len(reccomendationsRetail.index)

26736

In [83]:
reccomendationsRetail

,session_id,reccomended
0,2,"[9750, 1949, 42535, 35902, 11704, 48174, 64530..."
1,19,"[15900, 22302, 19614, 3447, 43420, 25932, 6148..."
2,20,"[6210, 27419, 68635, 59354, 61480, 60681, 2593..."
3,24,"[15900, 22302, 19614, 3447, 43420, 25932, 6148..."
4,25,"[892, 22302, 19614, 3447, 17288, 43420, 61480,..."
...,...,...
731,195468,"[15900, 22302, 19614, 3447, 43420, 25932, 6148..."
732,195471,"[15900, 22302, 19614, 3447, 43420, 25932, 6148..."
733,195501,"[28748, 51859, 3334, 60792, 31192, 33621, 3007..."
734,195503,"[15900, 22302, 19614, 3447, 43420, 25932, 6148..."


In [101]:
reccomendationsRetail.loc[reccomendationsRetail['session_id']==25]['reccomended'].values[0]

[892, 22302, 19614, 3447, 17288, 43420, 61480, 60681]

In [110]:
### Hit rate calculations:
hits3 = 0
total3 = 0
hits5 = 0
total5 = 0
hits8 = 0
total8 = 0
for index, row in sorted_events.iterrows():
  if index % 2000 == 0:
    print('Progress = ' + str(index/len(sorted_events)))
  reccomendedVales = reccomendationsRetail.loc[reccomendations['session_id']==row['session_id']].reccomended.values[0]
  if row['item_id'] in reccomendedVales[:3]:
    hits3 += 1
    total3 += 1
  else:
    total3 += 1
  if row['item_id'] in reccomendedVales[:5]:
    hits5 += 1
    total5 += 1
  else:
    total5 += 1
  if row['item_id'] in reccomendedVales[:8]:
    hits8 += 1
    total8 += 1
  else:
    total8 += 1
print('Hit rate at 3'+ ':' + str(hits3/total3))
print('Hit rate at 5'+ ':' + str(hits5/total5))
print('Hit rate at 8'+ ':' + str(hits8/total8))

Progress = 0.0
Progress = 0.03514629645901063
Progress = 0.07029259291802126
Progress = 0.1054388893770319
Progress = 0.14058518583604251
Progress = 0.17573148229505317
Progress = 0.2108777787540638
Progress = 0.24602407521307443
Progress = 0.28117037167208503
Progress = 0.3163166681310957
Progress = 0.35146296459010634
Progress = 0.38660926104911697
Progress = 0.4217555575081276
Progress = 0.4569018539671382
Progress = 0.49204815042614886
Progress = 0.5271944468851595
Progress = 0.5623407433441701
Progress = 0.5974870398031807
Progress = 0.6326333362621914
Progress = 0.667779632721202
Progress = 0.7029259291802127
Progress = 0.7380722256392233
Progress = 0.7732185220982339
Progress = 0.8083648185572445
Progress = 0.8435111150162552
Progress = 0.8786574114752658
Progress = 0.9138037079342765
Progress = 0.948950004393287
Progress = 0.9840963008522977
Hit rate at 3:0.1573499692469906
Hit rate at 5:0.17160179246111942
Hit rate at 8:0.18198752306475705


In [139]:
reccomendationsRetail3 = pd.DataFrame()
startidx = 0
endidx = 1000
while startidx < len(usersdf.index):
  batch = sarsRetail.recommend_k_items(usersdf[startidx:endidx],3)
  reccomendationsRetail3 = pd.concat([batch,reccomendationsRetail3])
  startidx = endidx
  endidx += 1000

reccomendationsRetail5 = pd.DataFrame()
startidx = 0
endidx = 1000
while startidx < len(usersdf.index):
  batch = sarsRetail.recommend_k_items(usersdf[startidx:endidx],5)
  reccomendationsRetail5 = pd.concat([batch,reccomendationsRetail5])
  startidx = endidx
  endidx += 1000

reccomendationsRetail8 = pd.DataFrame()
startidx = 0
endidx = 1000
while startidx < len(usersdf.index):
  batch = sarsRetail.recommend_k_items(usersdf[startidx:endidx],8)
  reccomendationsRetail8 = pd.concat([batch,reccomendationsRetail8])
  startidx = endidx
  endidx += 1000

In [150]:
## Ncdg:

import recommenders.evaluation.python_evaluation as evalMetrics

ndcg3 = evalMetrics.ndcg_at_k(usersdf, reccomendationsRetail3,'session_id','item_id','prediction', k = 3)
ndcg5 = evalMetrics.ndcg_at_k(usersdf, reccomendationsRetail5,'session_id','item_id','prediction', k = 5)
ndcg8 = evalMetrics.ndcg_at_k(usersdf, reccomendationsRetail8,'session_id','item_id','prediction', k = 8)

print('NDCG at 3: ' + str(ndcg3))
print('NDCG at 5: ' + str(ndcg5))
print('NDCG at 8: ' + str(ndcg8))

NDCG at 3: 0.18358768523058877
NDCG at 5: 0.18616771186644382
NDCG at 8: 0.18736916893900224
